# Google Forms

In [1]:
# https://forms.gle/VP7qS5F3GzU2QLBRA
# https://docs.google.com/forms/d/e/1FAIpQLSftqW9lIXy-g9IwpVfGGlDlYS-BKmxsSBb5_9o9MYfX6VB9ow/viewform?usp=sf_link

# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# Viewing a form - working 
from googleapiclient import discovery
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import os.path

SCOPES = [
    "https://www.googleapis.com/auth/forms.body",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file"
]
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("client_secrets.json", SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())
    
# Creating form service
form_service = discovery.build("forms", "v1", credentials=creds,)
sheets_service = discovery.build("sheets", "v4", credentials=creds)


In [2]:

# Prints the title of the sample form:
form_id = "14vM9XQp7BgHLA6JjtasZoxlFlGfRJdlqrRZqhcw9XHs"
result = form_service.forms().get(formId=form_id).execute()
print(result)


# Now create a google form 

{'formId': '14vM9XQp7BgHLA6JjtasZoxlFlGfRJdlqrRZqhcw9XHs', 'info': {'title': 'Python A Testing', 'description': 'Description ', 'documentTitle': 'Python A Testing'}, 'settings': {}, 'revisionId': '0000001f', 'responderUri': 'https://docs.google.com/forms/d/e/1FAIpQLSftqW9lIXy-g9IwpVfGGlDlYS-BKmxsSBb5_9o9MYfX6VB9ow/viewform', 'items': [{'itemId': '6db7d7a2', 'title': 'Untitled Question 1', 'questionItem': {'question': {'questionId': '4ce28e38', 'required': True, 'choiceQuestion': {'type': 'CHECKBOX', 'options': [{'value': 'Option 1'}, {'value': 'Option 2'}]}}}}, {'itemId': '0275da25', 'title': 'Question 2 ', 'questionItem': {'question': {'questionId': '3299072c', 'choiceQuestion': {'type': 'DROP_DOWN', 'options': [{'value': 'Option 1'}, {'value': 'Option 2'}, {'value': 'Option 3'}]}}}}, {'itemId': '1da8df20', 'title': 'Question 3 (linear scale) ', 'questionItem': {'question': {'questionId': '76e521e5', 'scaleQuestion': {'low': 1, 'high': 5}}}}, {'itemId': '384f587f', 'questionGroupItem'

In [5]:
# *  Creating a form with some questions 


# Create the form with just the title
form = {
    "info": {
        "title": "Sample Form"
    }
}

created_form = form_service.forms().create(body=form).execute()

# Define the batch update request to add questions
form_questions = [
    {
        "createItem": {
            "item": {
                "title": "What is your name?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "textQuestion": {}
                    }
                }
            },
            "location": {
                "index": 0
            }
        }
    },
    {
        "createItem": {
            "item": {
                "title": "What is your favorite color?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "Red"},
                                {"value": "Blue"},
                                {"value": "Green"}
                            ]
                        }
                    }
                }
            },
            "location": {
                "index": 1
            }
        }
    }
]

batch_update_request = {
    "requests": form_questions
}

# Execute the batch update
batch_update_response  = form_service.forms().batchUpdate(formId=created_form['formId'], body=batch_update_request).execute()
print('batch_update_reponse', batch_update_response)

# Print the form ID and URL
print(f"Form ID: {created_form['formId']}")
print(f"Form URL: https://docs.google.com/forms/d/{created_form['formId']}/edit")


batch_update_reponse {'replies': [{'createItem': {'itemId': '77713cf4', 'questionId': ['53745c7a']}}, {'createItem': {'itemId': '10d0361b', 'questionId': ['2604b423']}}], 'writeControl': {'requiredRevisionId': '00000004'}}
Form ID: 1SWOV9lAlfN9vPwC03v25vMNgm6t22_B7iZ6tBUR-0ZQ
Form URL: https://docs.google.com/forms/d/1SWOV9lAlfN9vPwC03v25vMNgm6t22_B7iZ6tBUR-0ZQ/edit


# MANUAL 
Add some reponses to the form using web browser and another email

In [6]:
# Viewing reposes - Working 
import pprint
# Define the form ID
form_id = created_form['formId']

# Retrieve form responses
responses = form_service.forms().responses().list(formId=form_id).execute()

print("Responses:")
pprint.pprint(responses)

Responses:
{'responses': [{'answers': {'2604b423': {'questionId': '2604b423',
                                         'textAnswers': {'answers': [{'value': 'Red'}]}},
                            '53745c7a': {'questionId': '53745c7a',
                                         'textAnswers': {'answers': [{'value': 'asdf '}]}}},
                'createTime': '2024-09-26T23:09:10.739Z',
                'lastSubmittedTime': '2024-09-26T23:09:10.739551Z',
                'responseId': 'ACYDBNhRz83Z2b8Kw2Oz1pfcpgzZk6gWYvLd3IwPswiMSSV6K4GTXRIMYPRYRu7a9ggVub4'}]}


In [7]:
# Create a new Google Sheet
spreadsheet = {
    "properties": {
        "title": "Form Responses"
    }
}
created_sheet = sheets_service.spreadsheets().create(body=spreadsheet).execute()


In [8]:

# ! Link the form to the Google Sheet ! This is not working 
form_update = {
    "requests": [
        {
            "updateSettings": {
                "settings": {
                    "responseDestination": {
                        "destinationType": "SPREADSHEET",
                        "spreadsheetId": created_sheet['spreadsheetId']
                    }
                },
                "updateMask": "responseDestination"
            }
        }
    ]
}
form_service.forms().batchUpdate(formId=created_form['formId'], body=form_update).execute()

# Print the form ID, form URL, and sheet URL
print(f"Form ID: {created_form['formId']}")
print(f"Form URL: https://docs.google.com/forms/d/{created_form['formId']}/edit")
print(f"Sheet URL: https://docs.google.com/spreadsheets/d/{created_sheet['spreadsheetId']}/edit")

HttpError: <HttpError 400 when requesting https://forms.googleapis.com/v1/forms/1SWOV9lAlfN9vPwC03v25vMNgm6t22_B7iZ6tBUR-0ZQ:batchUpdate?alt=json returned "Invalid JSON payload received. Unknown name "responseDestination" at 'requests[0].update_settings.settings': Cannot find field.". Details: "[{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'requests[0].update_settings.settings', 'description': 'Invalid JSON payload received. Unknown name "responseDestination" at \'requests[0].update_settings.settings\': Cannot find field.'}]}]">




# Microsoft Forms 

In [1]:
# Getting access token for Microsoft Graph API - working
# App, access_token
# !pip install msal
import msal
import os 
# Define the necessary parameters
client_secret = os.getenv("CLIENT_SECRET")
secret_id = os.getenv("SECRET_ID")
app_id = os.getenv("APP_ID")
tenant_id = os.getenv("TENANT_ID")
authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = ["https://graph.microsoft.com/.default"]

# Create a confidential client application
app = msal.ConfidentialClientApplication(
    client_id=app_id,
    client_credential=client_secret,
    authority=authority
)

# Acquire a token
result = app.acquire_token_for_client(scopes=scopes)

# Handle the response
if "access_token" in result:
    access_token = result["access_token"]
    print("Access Token:", access_token)
else:
    print("Failed to get access token:", result.get("error"), result.get("error_description"))

Access Token: eyJ0eXAiOiJKV1QiLCJub25jZSI6ImlpLURGd3lkSm9ZczVEQWp4Y2FZR3NKYnRfNGxra2xReHJXSUJGUWtqb2MiLCJhbGciOiJSUzI1NiIsIng1dCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyIsImtpZCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8zM2FhZDg3Yy0xNzQyLTQwMjQtOWVlZS04ZTg5Yjc4ZmQ5OGQvIiwiaWF0IjoxNzI3Mjk5MzM2LCJuYmYiOjE3MjcyOTkzMzYsImV4cCI6MTcyNzMwMzIzNiwiYWlvIjoiRTJkZ1lNaVBGMXpFcm1HMEtHbG42cHIzVys0eUFBQT0iLCJhcHBfZGlzcGxheW5hbWUiOiJjYW52YXMgYXV0b21hdGlubyIsImFwcGlkIjoiMjRmYjdkMWQtMTk5Zi00N2UyLWE4ODItYzZiZGVmODg2Nzk1IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMzNhYWQ4N2MtMTc0Mi00MDI0LTllZWUtOGU4OWI3OGZkOThkLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMDM3ZTEwMzUtNTI4NS00ZWFlLWJkOGItYWNiNzA5NzZlMmE3IiwicmgiOiIwLkFXOEJmTmlxTTBJWEpFQ2U3bzZKdDRfWmpRTUFBQUFBQUFBQXdBQUFBQUFBQUFCdkFRQS4iLCJzdWIiOiIwMzdlMTAzNS01Mjg1LTRlYWUtYmQ4Yi1hY2I3MDk3NmUyYTciLCJ0ZW5hbnRfcmVnaW9uX3Njb3BlIjoiTkEiLCJ0aWQiOiIzM2FhZDg3Yy0xNzQyLTQwM

In [ ]:
# trying to read the files in the onedrive
import requests 

# Step 2: Use the access token to access OneDrive
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
}

# Example: List files in the root of your OneDrive
graph_api_endpoint = 'https://graph.microsoft.com/v1.0/me/drive/root/children'
response = requests.get(graph_api_endpoint, headers=headers)

if response.status_code == 200:
    files = response.json().get('value', [])
    for file in files:
        print(f"Name: {file['name']}, Type: {file['file']['mimeType'] if 'file' in file else 'folder'}")
else:
    print(f"Error: {response.status_code}, {response.text}")


In [2]:
import requests
import msal
import os 
import jwt

def decode_jwt(token):
    return jwt.decode(token, options={"verify_signature": False})



# Define the necessary parameters
client_secret = os.getenv("CLIENT_SECRET")
secret_id = os.getenv("SECRET_ID")
app_id = os.getenv("APP_ID")
tenant_id = os.getenv("TENANT_ID")
authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = [    
        "https://graph.microsoft.com/.default",
        # "https://forms.office.com//Forms.Read.All",
        # "https://forms.cloud.microsoft//Responses.Read.All",
        # "https://graph.microsoft.com/User.Read", 
          
          ]
# Create a confidential client application
app = msal.ConfidentialClientApplication(
    client_id=app_id,
    client_credential=client_secret,
    authority=authority
)

# Acquire a token
# app.remove_tokens_for_client()
result = app.acquire_token_for_client(scopes=scopes)

# Handle the response
if 'access_token' not in result:
    print("Failed to get access token:", result.get("error"), result.get("error_description"))
    raise Exception("Failed to get access token")

access_token = result["access_token"]
print("Access Token:", access_token)
decoded_token = decode_jwt(access_token)
print("Token claims:", decoded_token)
token_scopes = decoded_token.get('scp', '').split()
print("Token scopes:", token_scopes)
print("Required scopes present:", all(scope in token_scopes for scope in scopes))
# print("Graph API endpoint:", graph_api_endpoint)



form_url = "https://forms.office.com/Pages/ResponsePage.aspx?id=DQSIkWdsW0yxEjajBLZtrQAAAAAAAAAAAAMAANMPSuxUQTJTVDdFUE5JTEk4OEZCRDRBTDlISEMzTi4u"
### Getting the form id based on the form url
from urllib.parse import urlparse, parse_qs
parsed_url = urlparse(form_url)
query_params = parse_qs(parsed_url.query)
form_id = query_params['id'][0]
print(f"Form ID: {form_id}")


# Use the /users endpoint to get user details
graph_api_endpoint = "https://graph.microsoft.com/v1.0/users"
headers = {'Authorization': 'Bearer ' + access_token}
response = requests.get(graph_api_endpoint, headers=headers)
# Debugging information
print(f"Response Status Code: {response.status_code}")
print(f"Response Text: {response.text}")

if response.status_code != 200:
    print(f"Error: {response.status_code}, {response.text}")
    raise Exception("Failed to get user details")

user_details = response.json()
user_id = user_details['value'][0]['id']  # Assuming you want the first user
print(f"User ID: {user_id}")


forms_api_endpoint = f"https://forms.office.com/formapi/api/{tenant_id}/users/{user_id}/light/forms('{form_id}')?$select=id,title,questions&$expand=questions($expand=choices)"
headers = {'Authorization': 'Bearer ' + access_token}
response = requests.get(forms_api_endpoint, headers=headers)


if response.status_code == 200:
    form_details = response.json()
    print(f"Form ID: {form_details['id']}")
    print(f"Title: {form_details['title']}")
    for question in form_details['questions']:
        print(f"Question: {question['title']}")
        if 'choices' in question:
            for choice in question['choices']:
                print(f" - Choice: {choice['text']}")
else:
    print(f"Error: {response.status_code}, {response.text}")


Access Token: eyJ0eXAiOiJKV1QiLCJub25jZSI6ImFvSzV3NjFCVTJhaHhld2otczhwZk5vTzhRZ3lhY1VoNUp2RmVLcGRPbmMiLCJhbGciOiJSUzI1NiIsIng1dCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyIsImtpZCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8zM2FhZDg3Yy0xNzQyLTQwMjQtOWVlZS04ZTg5Yjc4ZmQ5OGQvIiwiaWF0IjoxNzI3MzA1MzUzLCJuYmYiOjE3MjczMDUzNTMsImV4cCI6MTcyNzMwOTI1MywiYWlvIjoiRTJkZ1lMQlBubXJwZlcwbDY5WjduWTRyTm9xcUFRQT0iLCJhcHBfZGlzcGxheW5hbWUiOiJjYW52YXMgYXV0b21hdGlubyIsImFwcGlkIjoiMjRmYjdkMWQtMTk5Zi00N2UyLWE4ODItYzZiZGVmODg2Nzk1IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMzNhYWQ4N2MtMTc0Mi00MDI0LTllZWUtOGU4OWI3OGZkOThkLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMDM3ZTEwMzUtNTI4NS00ZWFlLWJkOGItYWNiNzA5NzZlMmE3IiwicmgiOiIwLkFXOEJmTmlxTTBJWEpFQ2U3bzZKdDRfWmpRTUFBQUFBQUFBQXdBQUFBQUFBQUFCdkFRQS4iLCJzdWIiOiIwMzdlMTAzNS01Mjg1LTRlYWUtYmQ4Yi1hY2I3MDk3NmUyYTciLCJ0ZW5hbnRfcmVnaW9uX3Njb3BlIjoiTkEiLCJ0aWQiOiIzM2FhZDg3Yy0xNzQyLTQwM

Exception: Failed to get user details

In [ ]:
# https://mofumofupower.hatenablog.com/entry/formsapi

In [14]:
import msal
import requests
import webbrowser
from urllib.parse import urlparse, parse_qs

# Your app registration details
client_secret = os.getenv("CLIENT_SECRET")
secret_id = os.getenv("SECRET_ID")
app_id = os.getenv("APP_ID")
tenant_id = os.getenv("TENANT_ID")

redirect_uri = "http://localhost:8000"

# MSAL configuration
authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = ["User.Read", "Forms.Read.All", "Forms.ReadWrite.All"]

# Create the MSAL app
app = msal.PublicClientApplication(app_id, authority=authority, redirect_uri=redirect_uri)

# Get the authorization URL
auth_url = app.get_authorization_request_url(scopes)
print(f"Please go to this URL and authorize the application: {auth_url}")

# Open the authorization URL in the web browser
webbrowser.open(auth_url)

# After the user logs in and authorizes the application, they will be redirected to the redirect URI with a code
# You need to capture the code from the redirect URI
redirect_response = input("Paste the full redirect URL here: ")
parsed_url = urlparse(redirect_response)
code = parse_qs(parsed_url.query).get('code', [None])[0]

# Acquire token using the authorization code
result = app.acquire_token_by_authorization_code(code, scopes=scopes)

if "access_token" in result:
    # Get user ID
    graph_api_endpoint = "https://graph.microsoft.com/v1.0/me"
    headers = {'Authorization': 'Bearer ' + result['access_token']}
    response = requests.get(graph_api_endpoint, headers=headers)

    if response.status_code == 200:
        user_details = response.json()
        user_id = user_details['id']
        print(f"User ID: {user_id}")

        # Use the access token to call Microsoft Forms API
        form_url = "https://forms.office.com/Pages/ResponsePage.aspx?id=DQSIkWdsW0yxEjajBLZtrQAAAAAAAAAAAAMAANMPSuxUQTJTVDdFUE5JTEk4OEZCRDRBTDlISEMzTi4u"
        # Getting the form id based on the form url
        parsed_url = urlparse(form_url)
        query_params = parse_qs(parsed_url.query)
        form_id = query_params.get('id', [None])[0]

        print(f"Form ID: {form_id}")

        forms_api_endpoint = f"https://forms.office.com/formapi/api/{tenant_id}/users/{user_id}/light/forms('{form_id}')?$select=id,title,questions&$expand=questions($expand=choices)"
        headers = {'Authorization': 'Bearer ' + result['access_token']}
        response = requests.get(forms_api_endpoint, headers=headers)

        if response.status_code == 200:
            form_details = response.json()
            print(f"Form ID: {form_details['id']}")
            print(f"Title: {form_details['title']}")
            for question in form_details['questions']:
                print(f"Question: {question['title']}")
                if 'choices' in question:
                    for choice in question['choices']:
                        print(f" - Choice: {choice['text']}")
        else:
            print(f"Error: {response.status_code}, {response.text}")
    else:
        print(f"Error: {response.status_code}, {response.text}")
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))

Please go to this URL and authorize the application: https://login.microsoftonline.com/33aad87c-1742-4024-9eee-8e89b78fd98d/oauth2/v2.0/authorize?client_id=24fb7d1d-199f-47e2-a882-c6bdef886795&response_type=code&scope=https%3A%2F%2Fgraph.microsoft.com%2F.default+offline_access+openid+profile
invalid_request
AADSTS900144: The request body must contain the following parameter: 'code'. Trace ID: 12e2b297-8c33-4532-9c4f-6e50e585f200 Correlation ID: 4d2deb9a-a942-4f25-a4b9-d05db719934c Timestamp: 2024-09-25 21:22:48Z
4d2deb9a-a942-4f25-a4b9-d05db719934c


In [9]:
import webbrowser
SCOPES = ['User.Read']
REDIRECT_URI = 'http://localhost'

authority = f'https://login.microsoftonline.com/{tenant_id}'

client = msal.PublicClientApplication(client_id=app_id, authority=authority)

authorization_url = client.get_authorization_request_url(SCOPES, redirect_uri=REDIRECT_URI)
print("Open the following URL in a browser to authorize the application:")
print(authorization_url)
webbrowser.open(authorization_url)


Open the following URL in a browser to authorize the application:
https://login.microsoftonline.com/33aad87c-1742-4024-9eee-8e89b78fd98d/oauth2/v2.0/authorize?client_id=24fb7d1d-199f-47e2-a882-c6bdef886795&response_type=code&redirect_uri=http%3A%2F%2Flocalhost&scope=User.Read+offline_access+openid+profile


True

In [10]:

from urllib.parse import urlparse, parse_qs
url = "http://localhost/?code=0.AW8BfNiqM0IXJECe7o6Jt4_ZjR19-ySfGeJHqILGve-IZ5VvAZA.AgABBAIAAAApTwJmzXqdR4BN2miheQMYAwDs_wUA9P8RpIXxXHWIIXmnD0_jy_u6aNPDmGVZMyeM5xWmVAN92V0dZ36jBkOzi8G67tsRKczaLpx_SpM5rQvLEV2te7BWwDGVLCzWDhsxYR21svmDDqYIhRbOyRqjQ3MFyCWqTRZ7dXpRlxXD8JbBOUWgXh9Qd20F7IB4fxURjUjiW-HH1ZmXwq3r3pbTATml7KyHVnyGawaxxclSToKt-z_ordNaaqhcxzU1dpD9uYuzoZ00GzRBTC9tB002LRiFYxzzj3SU4cNhCvLzInR_Bl45G-4FVtIad6EYAsNE1FJuRFOwdtwOScN03M4DGN6WiCLkjXBLskiZj2HhvBgnyeG5Bm4GCvZLZtgA3yVRov0ATShBTEaLz_EJ3jo_FLw6onX3kb5KE0hyFxP4ZuFBLH8P7JC53a6unVx5bk1B_G2w5nLoeGKbafH5vrYcndIbjuzAAj2nQG4cIMOgKV3ihiTHmqx-AIyWJRRa61ORV604850wQ3DIoHk5v_mBkKG2vXdtwzvO7K-6FpZcf7gtdhJXmj5kOu3rVgHNPY7sEBbUHft4OwNOzZPlFIRc-I2uzxR-EiaXp17Qf9BdqviudnT5ZqbL2JDWDhgg0aBAcJCPr3jn5nsAkTKDfzF2w8PFP0qCIDHZlOsRR8nBfjAxwmelRXP5GGHKEMSOo-7NFhH_sA6elWIIetqrYAWq_lUp3jVPcOV0vxopQftF8fcQ_BsN7q5x8YfQ3I2CzYReZGeDv45apcWcR0t5449lpz8Ge27APT-Sxro9tCDpnpGymsvlUqF2UHkS4YZEjFACdwHiVXkHlW8YVeNbi3OQToFS37bg6r3dkKGchIbgurLAgrZszSG5bsuUXutCOqASJm_8ISk-NXEeflxAzIz_EvF5olV1nOJLbYNBdsCAtrB21VobgKxs-udfeotHm0jlmrJ4S2xNdV5r77y3oOXhLK7kEm1IolbSCIP235z64pQlCNWgGx4oeOfC5R1LI-f1fGF9z0MdCFNUgLiVDZ3La12i_F97N0Wq7fBvvq7lZECocAckdw05eFcAeuJKP-LYYlhpbbsnXc3XkgYRUp0PgVwa_26ODznWdcXhl4ziKST8moE5Zqbaq0Wh5kBYNA-Icc7K6CZYghFRkU_VfjqtpUj-aACjSQj9SsNTOVTTizrkYJJE7AtPjovvPkuNUhJ6Svt7l1pU1CouvsUQlClKaHMn5h-Y4fvRmGDg2_y3O8GfMPkfS8tzvgwGp5wewpGDWg12DfH8kH8Smqv38TDSo9mrv2Pl4CHAYYdGLNQhRCzJWJw9j3YLmvxEjPJ13FqhwpYRFfT4anqAFVLginP4WhLcC2_gdSvJsLwbxfnx9VYf7_1Yhcts1v3lLLyIzPpjfswNWi33d3Yhxk4DfZdpvTCjOvWSPb8uOVe_CkOajz4g&session_state=c6bfde7c-9fc6-4bfb-8a84-f8720243b0e4"
parsed_url = urlparse(url)
query_params = parse_qs(parsed_url.query)
auth_code = query_params.get('code', [None])[0]
print(auth_code)

authorization_code = auth_code
access_token = client.acquire_token_by_authorization_code(code=authorization_code, scopes=SCOPES)
print(access_token)
access_id = access_token['access_token']

0.AW8BfNiqM0IXJECe7o6Jt4_ZjR19-ySfGeJHqILGve-IZ5VvAZA.AgABBAIAAAApTwJmzXqdR4BN2miheQMYAwDs_wUA9P8RpIXxXHWIIXmnD0_jy_u6aNPDmGVZMyeM5xWmVAN92V0dZ36jBkOzi8G67tsRKczaLpx_SpM5rQvLEV2te7BWwDGVLCzWDhsxYR21svmDDqYIhRbOyRqjQ3MFyCWqTRZ7dXpRlxXD8JbBOUWgXh9Qd20F7IB4fxURjUjiW-HH1ZmXwq3r3pbTATml7KyHVnyGawaxxclSToKt-z_ordNaaqhcxzU1dpD9uYuzoZ00GzRBTC9tB002LRiFYxzzj3SU4cNhCvLzInR_Bl45G-4FVtIad6EYAsNE1FJuRFOwdtwOScN03M4DGN6WiCLkjXBLskiZj2HhvBgnyeG5Bm4GCvZLZtgA3yVRov0ATShBTEaLz_EJ3jo_FLw6onX3kb5KE0hyFxP4ZuFBLH8P7JC53a6unVx5bk1B_G2w5nLoeGKbafH5vrYcndIbjuzAAj2nQG4cIMOgKV3ihiTHmqx-AIyWJRRa61ORV604850wQ3DIoHk5v_mBkKG2vXdtwzvO7K-6FpZcf7gtdhJXmj5kOu3rVgHNPY7sEBbUHft4OwNOzZPlFIRc-I2uzxR-EiaXp17Qf9BdqviudnT5ZqbL2JDWDhgg0aBAcJCPr3jn5nsAkTKDfzF2w8PFP0qCIDHZlOsRR8nBfjAxwmelRXP5GGHKEMSOo-7NFhH_sA6elWIIetqrYAWq_lUp3jVPcOV0vxopQftF8fcQ_BsN7q5x8YfQ3I2CzYReZGeDv45apcWcR0t5449lpz8Ge27APT-Sxro9tCDpnpGymsvlUqF2UHkS4YZEjFACdwHiVXkHlW8YVeNbi3OQToFS37bg6r3dkKGchIbgurLAgrZszSG5bsuUXutCOqASJm_8ISk-NXEeflxAzIz_EvF5olV1nOJLbYNBds

KeyError: 'access_token'

0.AW8BfNiqM0IXJECe7o6Jt4_ZjR19-ySfGeJHqILGve-IZ5VvAZA.AgABBAIAAAApTwJmzXqdR4BN2miheQMYAwDs_wUA9P9XDkf14afxHn-oxoGw2oS0W92RUvHsjO4EOKjU7iejunvds3zZZ8oGKhjDecDffJsSEeJ3T79IUbnOwFF24Bzx1L5Ld7R1W8q1GlnHHThvCs1Q7Y58RnPL5Jt4vdAEWE_Yg3TW79-und-x_Euqyikv364IsaNr21RYLxkJw9oEGHrskxPFE_hEjDNw7WLyGFUunwzQTmVPAXdu_tX-enjDWtJvdTZJfshF0iQ5tapevW8bzmds3Hrixhk6Smp6SxsDNsb4XA07iDz_tNaojyrivUAzwiP9xqcF7PVR6JJz9kEuLYtgjZO7KTvGAbVk-Y52pNfchQUk9iPYIZWyWeFIYwYeSbzSH4p4UDhXdDkO0USmih49HAiwLlhwM7EwGZLZxHARijdmfIcGxjxnJMee_z5UkyGVKldLIhFnom374ENfQOTbpG0_foGTtrnxSyfIYlDIMLOvGElXkw0WpfnrJBI5xOpO1d9jZ6qZ08N9ultmEsaWzmJk1VlJXPEvtp2yrFrQR7gKJqtGCv2LOZ0ap9Yy6wUfS52NxfXOT1rs9kx7_E2lMGUc2cQd0ghdixyMr5JJxj7ass0z-PD6eULIdrt76MACxC2xswNGbsr2X6u27467YpUJOI0eqxYwzEKg2PDWvybJ7Wsqr_rgzJ5FeatQxdZ-YP7WnUJUwNDJsM69_ogLCvMY3soaFPQP49EG55INf1uOpDsLQZRi2ndqIZ-Timlvvssor4frYG5TJCjbldSN7gTmUPvMrPFHUmEr6zyp-HoJ9FClUsUmaaJGVpCBWA5fJrtIrmj9cohHremjdP3zAJtUygN5-sSI76dcyKIhePkmiK4YJMenCPAFBae7ZwoE7-lPZw8RyU7XfaNY9n6jHVonwz98_9pCR15UyUcafE

{'error': 'invalid_client', 'error_description': "AADSTS700025: Client is public so neither 'client_assertion' nor 'client_secret' should be presented. Trace ID: 0130a6a0-0ffe-41af-b071-37f9c7e92f00 Correlation ID: 9cd6847e-fed4-4c00-bcf9-1c007e54060a Timestamp: 2024-09-25 23:10:00Z", 'error_codes': [700025], 'timestamp': '2024-09-25 23:10:00Z', 'trace_id': '0130a6a0-0ffe-41af-b071-37f9c7e92f00', 'correlation_id': '9cd6847e-fed4-4c00-bcf9-1c007e54060a'}


KeyError: 'access_token'

In [ ]:
import msal

def generate_access_token():
    access_token_cache = msal.SerializableTokenCache()
    # read the token file 
    if os.path.exists("token.json"):
        access_token_cache.deserialize(open("token.json", "r").read())
        
    
    client = msal.PublicClientApplication (cLient_id=app_id, token_cache=access_token_cache)
    
    accounts = client.get_accounts()
    if accounts:
        token_response = client.acquire_token_silent(scopes, accounts[0])
    token_response = client.acquire_token_silent(SCOPES, account=accounts[0])
    pr